In [ ]:
import os

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.1/723.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import datetime
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold

In [ ]:
import cv2

In [ ]:
import torch

In [ ]:
list_input_dir = ['/content/gdrive/My Drive/ES100root/masks','/content/gdrive/My Drive/ES100root/masks_patient/P5','/content/gdrive/My Drive/ES100root/masks_patient/P7', '/content/gdrive/My Drive/ES100root/masks_patient/P8', '/content/gdrive/My Drive/ES100root/masks_patient/P9' ]
outdir= '/content/gdrive/My Drive/ES100_Kfold/labels'

In [ ]:
for indir in list_input_dir:
  for j in os.listdir(indir):
      image_path = os.path.join(indir, j)
      # load the binary mask and get its contours
      if "frame" in image_path:
        mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)
        print(image_path)

        H, W = mask.shape
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # convert the contours to polygons
        polygons = []
        for cnt in contours:
            if cv2.contourArea(cnt) > 200:
                polygon = []
                for point in cnt:
                    x, y = point[0]
                    polygon.append(x / W)
                    polygon.append(y / H)
                polygons.append(polygon)
        with open('{}.txt'.format(os.path.join(outdir, j)[:-4]), 'w') as f:
          for polygon in polygons:
              for p_, p in enumerate(polygon):
                  if p_ == len(polygon) - 1:
                      f.write('{}\n'.format(p))
                  elif p_ == 0:
                      f.write('0 {} '.format(p))
                  else:
                      f.write('{} '.format(p))

          f.close()

/content/gdrive/My Drive/ES100root/masks/P1_frame_000002.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000005.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000003.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000004.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000000.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000006.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000001.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000016.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000009.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000010.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000008.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000007.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000015.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000011.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000013.png
/content/gdrive/My Drive/ES100root/masks/P1_frame_000014.png
/content/gdrive/My Drive

In [ ]:
dataset_path = Path('/content/gdrive/My Drive/ES100root') # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("labels_s/*.txt")) # all data in 'labels'

In [ ]:
backgrounds = sorted(dataset_path.rglob("2024-03-18_5-Fold_Cross-val-backgrounds/*.png"))

In [ ]:
indx_back = [back.stem for back in backgrounds] # uses base filename as ID (no extension)
back_df = pd.DataFrame([], columns=cls_idx, index=indx_back)

In [ ]:


kfolds_back = list(kf.split(back_df))

In [ ]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df_back = pd.DataFrame(index=indx_back, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[back_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[back_df.iloc[val].index] = 'val'

IndexError: positional indexers are out-of-bounds

In [ ]:
yaml_file = '/content/gdrive/My Drive/ES100root/config.yaml'  # your data YAML with data directories and names dictionary
with open(yaml_file, 'r', encoding="utf8") as y:
    classes = yaml.safe_load(y)
    classes = classes['names']
cls_idx = sorted(classes)


In [ ]:
indx = [l.stem for l in labels] # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [ ]:
labels_df

,posterior ascending
P3_frame_000002,NaN
P3_frame_000003,NaN
P3_frame_000004,NaN
P3_frame_000005,NaN
P3_frame_000006,NaN
...,...
P9_frame_000216,NaN
P9_frame_000217,NaN
P9_frame_000218,NaN
P9_frame_000219,NaN


In [ ]:
for label in labels:
    lbl_counter = Counter()
    print(label)

    with open(label,'r') as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(' ')[0])] += 1

    labels_df.loc[label.stem] = lbl_counter[0]

labels_df = labels_df.fillna(0.0) # replace `nan` values with `0.0`

/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000002.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000003.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000004.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000005.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000006.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000007.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000008.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000009.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000010.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000011.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000012.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000013.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000014.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000015.txt
/content/gdrive/My Drive/ES100root/labels_s/P3_frame_000016.txt
/content/gdrive/My Drive/ES100root/label

K-FOLD DATASET SPLIT

In [ ]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)   # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [ ]:
folds = [f'split_{n}' for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f'split_{idx}'].loc[labels_df.iloc[train].index] = 'train'
    folds_df[f'split_{idx}'].loc[labels_df.iloc[val].index] = 'val'

In [ ]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1E-7)
    fold_lbl_distrb.loc[f'split_{n}'] = ratio

In [ ]:
supported_extensions = ['.jpg', '.jpeg', '.png']

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / 'images' / 'train').rglob(f"*{ext}")))
    images.extend(sorted((dataset_path / 'images' / 'val').rglob(f"*{ext}")))
    images.extend(sorted((dataset_path / 'image_temp' / 'train_temp').rglob(f"*{ext}")))
    images.extend(sorted((dataset_path / 'image_temp' / 'val_temp').rglob(f"*{ext}")))
    images.extend(sorted((dataset_path / 'labels_temp' / 'P8_images').rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f'{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val')
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'train' / 'labels').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'images').mkdir(parents=True, exist_ok=True)
    (split_dir / 'val' / 'labels').mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f'{split}_dataset.yaml'
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, 'w') as ds_y:
        yaml.safe_dump({
            'path': split_dir.as_posix(),
            'train': 'train',
            'val': 'val',
            'names': classes
        }, ds_y)

In [ ]:
sorted_image = sorted(images, key = lambda image: image.stem)

In [ ]:
sorted_image

[PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000000.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000001.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000002.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000003.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000004.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000005.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000006.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000007.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000008.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000009.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000010.png'),
 PosixPath('/content/gdrive/My Drive/ES100root/images/train/P3_frame_000011.png'),
 Pos

In [ ]:
label.stem

'P9_frame_000220'

In [ ]:
label_counter = 0
for image in sorted_image:
  if image.stem in folds_df.index:
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / 'images'
        lbl_to_path = save_path / split / k_split / 'labels'

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(labels[label_counter], lbl_to_path / labels[label_counter].name)
    print(f"Image inputted from '{image.name}' and label inputted from '{labels[label_counter].name}'.")
    label_counter += 1

Image inputted from 'P3_frame_000002.png' and label inputted from 'P3_frame_000002.txt'.
Image inputted from 'P3_frame_000003.png' and label inputted from 'P3_frame_000003.txt'.
Image inputted from 'P3_frame_000004.png' and label inputted from 'P3_frame_000004.txt'.
Image inputted from 'P3_frame_000005.png' and label inputted from 'P3_frame_000005.txt'.
Image inputted from 'P3_frame_000006.png' and label inputted from 'P3_frame_000006.txt'.
Image inputted from 'P3_frame_000007.png' and label inputted from 'P3_frame_000007.txt'.
Image inputted from 'P3_frame_000008.png' and label inputted from 'P3_frame_000008.txt'.
Image inputted from 'P3_frame_000009.png' and label inputted from 'P3_frame_000009.txt'.
Image inputted from 'P3_frame_000010.png' and label inputted from 'P3_frame_000010.txt'.
Image inputted from 'P3_frame_000011.png' and label inputted from 'P3_frame_000011.txt'.
Image inputted from 'P3_frame_000012.png' and label inputted from 'P3_frame_000012.txt'.
Image inputted from '

In [ ]:
save_path_background

PosixPath('/content/gdrive/My Drive/ES100root/2024-03-18_5-Fold_Cross-val-backgrounds')

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!scp -r /content/gdrive/MyDrive/ES100root/2024-03-18_5-Fold_Cross-val-backgrounds '/content/gdrive/My Drive/ES100root/'

cp: '/content/gdrive/MyDrive/ES100root/2024-03-18_5-Fold_Cross-val-backgrounds' and '/content/gdrive/My Drive/ES100root/2024-03-18_5-Fold_Cross-val-backgrounds' are the same file


In [ ]:
save_path_background = Path(dataset_path / f'{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val-backgrounds')
save_path_background.mkdir(parents=True, exist_ok=True)

for image in sorted_image:
  if image.stem not in folds_df.index:

    shutil.copy(image, save_path_background / image.name)
    print(f"Image '{image.name}' not in folds_df.")

Image 'P3_frame_000000.png' not in folds_df.
Image 'P3_frame_000001.png' not in folds_df.
Image 'P3_frame_000038.png' not in folds_df.
Image 'P3_frame_000039.png' not in folds_df.
Image 'P3_frame_000040.png' not in folds_df.
Image 'P3_frame_000047.png' not in folds_df.
Image 'P3_frame_000049.png' not in folds_df.
Image 'P3_frame_000052.png' not in folds_df.
Image 'P3_frame_000080.png' not in folds_df.
Image 'P3_frame_000081.png' not in folds_df.
Image 'P3_frame_000082.png' not in folds_df.
Image 'P3_frame_000083.png' not in folds_df.
Image 'P3_frame_000084.png' not in folds_df.
Image 'P3_frame_000085.png' not in folds_df.
Image 'P3_frame_000086.png' not in folds_df.
Image 'P3_frame_000087.png' not in folds_df.
Image 'P3_frame_000088.png' not in folds_df.
Image 'P3_frame_000089.png' not in folds_df.
Image 'P3_frame_000090.png' not in folds_df.
Image 'P3_frame_000091.png' not in folds_df.
Image 'P3_frame_000092.png' not in folds_df.
Image 'P3_frame_000093.png' not in folds_df.
Image 'P3_

Training YOLO with K-fold data spilts

In [ ]:
#weights_path = '/content/gdrive/My Drive/ES100root/runs/EXP1_134579/exp1_A100/exp1m_final/weights/best.pt'
model = YOLO('yolov8s-seg.pt', task = 'detect')

100%|██████████| 22.8M/22.8M [00:00<00:00, 311MB/s]


In [ ]:
ds_yamls_temp = ['/content/gdrive/MyDrive/ES100root/2024-03-18_5-Fold_Cross-val/split_1/split_1_dataset.yaml',
                 '/content/gdrive/MyDrive/ES100root/2024-03-18_5-Fold_Cross-val/split_2/split_2_dataset.yaml',
                 '/content/gdrive/MyDrive/ES100root/2024-03-18_5-Fold_Cross-val/split_3/split_3_dataset.yaml',
                 '/content/gdrive/MyDrive/ES100root/2024-03-18_5-Fold_Cross-val/split_4/split_4_dataset.yaml',
                 '/content/gdrive/MyDrive/ES100root/2024-03-18_5-Fold_Cross-val/split_5/split_5_dataset.yaml']

In [ ]:
import torch.optim as optim

# Define hyperparameters

# Instantiate optimizer with specified hyperparameters
optimizer = optim.AdamW(model.parameters(), lr0=0.00734, lrf=0.00605, momentum=0.97106, weight_decay = 0.00055, warmup_epochs = 3.06711, warmup_momentum =0.72107, box = 8.39131, cls = 0.49205,dfl=1.67383,hsv_h=0.01125,hsv_s=0.4879,hsv_v=0.4,degrees=0.0,translate=0.10116,scale=0.58559,shear=0,perspective=0,flipud=0,fliplr=0.54761,mosaic=1,mixup=0,copy_paste=0)

TypeError: AdamW.__init__() got an unexpected keyword argument 'lr0'

In [ ]:
import torch.optim as optim

In [ ]:
results = {}

ksplit = 5

# Define your additional arguments here
batch = 16
project = 'kfold_demo'
epochs = 10

for k in range(ksplit):
    dataset_yaml = ds_yamls_temp[k]
    model.train(data=dataset_yaml, cfg = '/content/best_hyperparameters-2.yaml', epochs=epochs, batch=batch, project=project, optimizer = 'AdamW')  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis

Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
engine/trainer: task=segment, mode=train, model=None, data=/content/gdrive/MyDrive/ES100root/2024-03-18_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=kfold_demo, name=train4, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=F

train: Scanning /content/gdrive/.shortcut-targets-by-id/1DgH6NEok81EWCSfsITjAjTQzQIDNN6z_/ES100root/2024-03-18_5-Fold_Cross-val/split_1/train/labels.cache... 922 images, 68 backgrounds, 0 corrupt: 100%|██████████| 990/990 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
val: Scanning /content/gdrive/.shortcut-targets-by-id/1DgH6NEok81EWCSfsITjAjTQzQIDNN6z_/ES100root/2024-03-18_5-Fold_Cross-val/split_1/val/labels.cache... 231 images, 17 backgrounds, 0 corrupt: 100%|██████████| 248/248 [00:00<?, ?it/s]


Plotting labels to kfold_demo/train4/labels.jpg... 
optimizer: AdamW(lr=0.00734, momentum=0.97106) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.00055), 76 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to kfold_demo/train4
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10        11G      1.184      1.376     0.5316      1.318         14        640: 100%|██████████| 62/62 [00:17<00:00,  3.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.16it/s]

                   all        247        244      0.958      0.971       0.99       0.74      0.971      0.973       0.99      0.719



10 epochs completed in 0.088 hours.
Optimizer stripped from kfold_demo/train8/weights/last.pt, 23.8MB
Optimizer stripped from kfold_demo/train8/weights/best.pt, 23.8MB

Validating kfold_demo/train8/weights/best.pt...
Ultralytics YOLOv8.1.29 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
YOLOv8s-seg summary (fused): 195 layers, 11779987 parameters, 0 gradients, 42.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.79it/s]


                   all        247        244      0.956      0.972       0.99       0.74      0.971      0.971       0.99      0.719
Speed: 0.3ms preprocess, 1.4ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to kfold_demo/train8


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!scp -r /content/kfold_demo_345789 '/content/gdrive/My Drive/ES100root/runs'

In [ ]:
folds_df.to_csv('fold_distribution', sep=',', index=True, encoding='utf-8')

In [ ]:
folds_df

,split_1,split_2,split_3,split_4,split_5
P1_frame_000000,train,train,train,val,train
P1_frame_000001,val,train,train,train,train
P1_frame_000002,train,train,train,val,train
P1_frame_000003,train,train,val,train,train
P1_frame_000004,train,train,train,train,val
...,...,...,...,...,...
P9_frame_000216,val,train,train,train,train
P9_frame_000217,train,train,val,train,train
P9_frame_000218,train,train,val,train,train
P9_frame_000219,train,train,train,val,train


In [ ]:
model_1m_split1 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_small/train/weights/best.pt')

source = '/content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4'

results_1m_split1 = model_1m(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/", name="exp2m_split1")

counter = 0
for r in results_1m_split1:
  if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]


      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      written = cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/exp2m_iou_split1/P8_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())
      print(written)
  counter += 1



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 3 posterior ascendings, 14.2ms
video 1/1 (frame 2/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 1 posterior ascending, 12.9ms
video 1/1 (frame 3/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 (no detections), 9.6ms
video 1/1 (frame 4/301) /content/gdrive/My Drive/ES100root/Vide

In [ ]:
model_1m_split3 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_134579/train222/weights/best.pt')

source = '/content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4'

results_1m_split3 = model_1m(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/", name="exp2m_split3")

counter = 0
for r in results_1m_split3:
  if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]


      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      written = cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/exp2m_iou_split3/P8_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())
      print(written)
  counter += 1



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 3 posterior ascendings, 15.0ms
video 1/1 (frame 2/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 1 posterior ascending, 10.3ms
video 1/1 (frame 3/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 (no detections), 13.8ms
video 1/1 (frame 4/301) /content/gdrive/My Drive/ES100root/Vid

In [ ]:
model_1m_split4= YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_134579/train222/weights/best.pt')

source = '/content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4'

results_1m_split4 = model_1m(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/", name="exp2m_split4")

counter = 0
for r in results_1m_split4:
  if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/exp2m_iou_split4/P8_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

  counter += 1



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 3 posterior ascendings, 10.4ms
video 1/1 (frame 2/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 1 posterior ascending, 10.0ms
video 1/1 (frame 3/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 (no detections), 9.7ms
video 1/1 (frame 4/301) /content/gdrive/My Drive/ES100root/Vide

In [ ]:
model_1m_split5= YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_134579/train2222/weights/best.pt')

source = '/content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4'

results_1m_split5= model_1m(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/", name="exp2m_split5")

counter = 0
for r in results_1m_split5:
  if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      written = cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/exp2m_iou_split5/P8_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())
      print(written)
  counter += 1



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 3 posterior ascendings, 13.9ms
video 1/1 (frame 2/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 1 posterior ascending, 10.3ms
video 1/1 (frame 3/301) /content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4: 384x640 (no detections), 9.9ms
video 1/1 (frame 4/301) /content/gdrive/My Drive/ES100root/Vide

In [ ]:
counter = 0
for r in results_1m:
  if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      written = cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/exp2m_iou/P8_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())
      print(written)

  counter += 1

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
net = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_134579/train2/weights/best.pt')

model_state_dict = net.state_dict()

torch.save(model_state_dict, 'split2_onlyweights.pt')


In [ ]:
from torchsummary import summary

In [ ]:
model1.info(detailed=True)

layer                                     name  gradient   parameters                shape         mu      sigma
    0                      model.0.conv.weight     False         1296        [48, 3, 3, 3]   -0.00157      0.119 torch.float32
    1                        model.0.bn.weight     False           48                 [48]       3.17      0.738 torch.float32
    2                          model.0.bn.bias     False           48                 [48]      0.173       2.04 torch.float32
    3                      model.1.conv.weight     False        41472       [96, 48, 3, 3]  -0.000604      0.025 torch.float32
    4                        model.1.bn.weight     False           96                 [96]       2.72      0.664 torch.float32
    5                          model.1.bn.bias     False           96                 [96]      0.322       1.13 torch.float32
    6                  model.2.cv1.conv.weight     False         9216       [96, 96, 1, 1]   -0.00371     0.0366 torch.float3

(331, 27285968, 0, 110.64632320000001)

In [ ]:
net.info(detailed=True)

layer                                     name  gradient   parameters                shape         mu      sigma
    0                      model.0.conv.weight     False         1296        [48, 3, 3, 3]  -0.000613       0.12 torch.float32
    1                        model.0.bn.weight     False           48                 [48]       3.15      0.744 torch.float32
    2                          model.0.bn.bias     False           48                 [48]      0.156       2.06 torch.float32
    3                      model.1.conv.weight     False        41472       [96, 48, 3, 3]  -0.000722     0.0297 torch.float32
    4                        model.1.bn.weight     False           96                 [96]       2.71      0.666 torch.float32
    5                          model.1.bn.bias     False           96                 [96]      0.308       1.15 torch.float32
    6                  model.2.cv1.conv.weight     False         9216       [96, 96, 1, 1]   -0.00421     0.0405 torch.float3

/usr/local/lib/python3.10/dist-packages/ultralytics/utils/torch_utils.py:250: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1760.)
  % (i, name, p.requires_grad, p.numel(), list(p.shape), p.mean(), p.std(), p.dtype)


YOLOv8m-seg summary: 331 layers, 27240227 parameters, 0 gradients, 110.4 GFLOPs


(331, 27240227, 0, 110.39150079999999)

In [ ]:
type(model_test)

ultralytics.models.yolo.model.YOLO

In [ ]:
summary('split2_onlyweights.pt',(3,640,640))

AttributeError: 'str' object has no attribute 'apply'

In [ ]:
model1 = YOLO('yolov8m-seg.pt')
model2 = torch.load('split2_onlyweights.pt')

In [ ]:
len(model1.info()['name'])

YOLOv8m-seg summary: 331 layers, 27285968 parameters, 0 gradients, 110.6 GFLOPs


TypeError: tuple indices must be integers or slices, not str

In [ ]:
len(model2.keys())

537

In [ ]:
for keynew, keyorg in zip_directory

In [ ]:
for key, value in zip(model2.keys(), model2.values()):
  print(key + " " + str(len(value)))

model.model.0.conv.weight 48
model.model.0.bn.weight 48
model.model.0.bn.bias 48
model.model.0.bn.running_mean 48
model.model.0.bn.running_var 48


TypeError: len() of a 0-d tensor

In [ ]:
summary(model2,(3,640,640))

AttributeError: 'collections.OrderedDict' object has no attribute 'apply'

In [ ]:
import string
from collections import OrderedDict

In [ ]:
new_dict_keys = OrderedDict()
key_mapping = OrderedDict()


for key in model2.keys():
  key_mapping[key] = key.replace("model.","",1)

for old_key, new_key in key_mapping.items():
    model2[new_key] = model2.pop(old_key)

model2.keys()

odict_keys(['model.0.conv.weight', 'model.0.bn.weight', 'model.0.bn.bias', 'model.0.bn.running_mean', 'model.0.bn.running_var', 'model.0.bn.num_batches_tracked', 'model.1.conv.weight', 'model.1.bn.weight', 'model.1.bn.bias', 'model.1.bn.running_mean', 'model.1.bn.running_var', 'model.1.bn.num_batches_tracked', 'model.2.cv1.conv.weight', 'model.2.cv1.bn.weight', 'model.2.cv1.bn.bias', 'model.2.cv1.bn.running_mean', 'model.2.cv1.bn.running_var', 'model.2.cv1.bn.num_batches_tracked', 'model.2.cv2.conv.weight', 'model.2.cv2.bn.weight', 'model.2.cv2.bn.bias', 'model.2.cv2.bn.running_mean', 'model.2.cv2.bn.running_var', 'model.2.cv2.bn.num_batches_tracked', 'model.2.m.0.cv1.conv.weight', 'model.2.m.0.cv1.bn.weight', 'model.2.m.0.cv1.bn.bias', 'model.2.m.0.cv1.bn.running_mean', 'model.2.m.0.cv1.bn.running_var', 'model.2.m.0.cv1.bn.num_batches_tracked', 'model.2.m.0.cv2.conv.weight', 'model.2.m.0.cv2.bn.weight', 'model.2.m.0.cv2.bn.bias', 'model.2.m.0.cv2.bn.running_mean', 'model.2.m.0.cv2.bn.

In [ ]:
model2_YOLO = YOLO('split2_onlyweights.pt')

KeyError: 'model'

In [ ]:

for param_tensor in model1.state_dict():
  if param_tensor in model2.keys():
    print(model1.state_dict()[param_tensor].size())

torch.Size([48, 3, 3, 3])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([])
torch.Size([96, 48, 3, 3])
torch.Size([96])
torch.Size([96])
torch.Size([96])
torch.Size([96])
torch.Size([])
torch.Size([96, 96, 1, 1])
torch.Size([96])
torch.Size([96])
torch.Size([96])
torch.Size([96])
torch.Size([])
torch.Size([96, 192, 1, 1])
torch.Size([96])
torch.Size([96])
torch.Size([96])
torch.Size([96])
torch.Size([])
torch.Size([48, 48, 3, 3])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([])
torch.Size([48, 48, 3, 3])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([])
torch.Size([48, 48, 3, 3])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([])
torch.Size([48, 48, 3, 3])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([])
torch.Size([192, 96, 3, 3])
torch.Size([192])
torch.Size([192])
torch.Size([192])
torch.Size([192])
torch.Size([])
torch

In [ ]:
model2.i

In [ ]:
new_state_dict = {}
for param_tensor in model2.keys():

    if model1.state_dict()[param_tensor].size() != model2[param_tensor].size():
        new_state_dict[key] = value
    else:
        print(key)

model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2.2.bias
model.model.22.cv4.2

In [ ]:
model1.load_state_dict(new_state_dict)

RuntimeError: Error(s) in loading state_dict for YOLO:
	size mismatch for model.model.22.cv3.0.2.weight: copying a param with shape torch.Size([1, 192, 1, 1]) from checkpoint, the shape in current model is torch.Size([80, 192, 1, 1]).
	size mismatch for model.model.22.cv3.0.2.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.model.22.cv3.1.2.weight: copying a param with shape torch.Size([1, 192, 1, 1]) from checkpoint, the shape in current model is torch.Size([80, 192, 1, 1]).
	size mismatch for model.model.22.cv3.1.2.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.model.22.cv3.2.2.weight: copying a param with shape torch.Size([1, 192, 1, 1]) from checkpoint, the shape in current model is torch.Size([80, 192, 1, 1]).
	size mismatch for model.model.22.cv3.2.2.bias: copying a param with shape torch.Size([1]) fr

In [ ]:
model_test = model1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if device.type == 'cuda':
    model_test.load_state_dict(model2)
else:
    model_test.load_state_dict(model2)

RuntimeError: Error(s) in loading state_dict for YOLO:
	size mismatch for model.model.22.cv3.0.2.weight: copying a param with shape torch.Size([1, 192, 1, 1]) from checkpoint, the shape in current model is torch.Size([80, 192, 1, 1]).
	size mismatch for model.model.22.cv3.0.2.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.model.22.cv3.1.2.weight: copying a param with shape torch.Size([1, 192, 1, 1]) from checkpoint, the shape in current model is torch.Size([80, 192, 1, 1]).
	size mismatch for model.model.22.cv3.1.2.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([80]).
	size mismatch for model.model.22.cv3.2.2.weight: copying a param with shape torch.Size([1, 192, 1, 1]) from checkpoint, the shape in current model is torch.Size([80, 192, 1, 1]).
	size mismatch for model.model.22.cv3.2.2.bias: copying a param with shape torch.Size([1]) fr

In [ ]:
models_split1_345789 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_345789/train4/weights/best.pt')
models_split2_345789 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_345789/train5/weights/best.pt')
models_split3_345789 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_345789/train6/weights/best.pt')
models_split4_345789 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_345789/train7/weights/best.pt')
models_split5_345789 = YOLO('/content/gdrive/My Drive/ES100root/runs/kfold_demo_345789/train8/weights/best.pt')

In [ ]:
sources = ['/content/gdrive/My Drive/ES100root/Videos/CDW_4.6.23_RUL_PA1_1fps.mp4','/content/gdrive/My Drive/ES100root/Videos/CDW_11.16.22_RUL_PA1_1fps.mp4','/content/gdrive/My Drive/ES100root/Videos/CFY_6.6.22_RUL_PA1_1fps.mp4',
           '/content/gdrive/My Drive/ES100root/Videos/CFY_6.13.22_RUL_PA1_1fps.mp4','/content/gdrive/My Drive/ES100root/Videos/LS_5.15.23_RUL_PA1_1fps.mp4','/content/gdrive/My Drive/ES100root/Videos/LS_5.24.22_RUL_PA1_1fps.mp4',
           '/content/gdrive/My Drive/ES100root/Videos/US_2.15.23_RUL_PA1_1fps.mp4']

patient_num = ['1','3','4','5','7','8','9']
patient_counter = 0
for source in sources:
  results_split1_345789 = models_split1_345789(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_345789", name="split1")  # generator of Results objects
  results_split2_345789 = models_split2_345789(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_345789", name="split2")
  results_split3_345789 = models_split3_345789(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_345789", name="split3")
  results_split4_345789 = models_split4_345789(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_345789", name="split4")
  results_split5_345789 = models_split5_345789(source, save=True, save_conf=True, project="/content/gdrive/My Drive/ES100root/runs/EXP2s_345789", name="split5")

  counter = 0
  for r in results_split1_345789:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_345789/split1_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1

  counter = 0
  for r in results_split2_345789:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_345789/split2_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1

  counter = 0
  for r in results_split3_345789:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_345789/split3_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1

  counter = 0
  for r in results_split4_345789:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_345789/split4_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1

  counter = 0
  for r in results_split5_345789:
    if r.masks != None:
      masks = r.masks.data
      boxes = r.boxes.data

      clss = boxes[:, 5]
      print("clss")
      print(clss)

      seg_classes = list(r.names.values())
      #EXTRACT A SINGLE MASK WITH ALL THE CLASSES
      obj_indices = torch.where(clss != -1)
      obj_masks = masks[obj_indices]
      obj_mask = torch.any(obj_masks, dim=0).int() * 255
      cv2.imwrite(str(f'/content/gdrive/My Drive/ES100root/runs/EXP2s_345789/split5_iou/P{patient_num[patient_counter]}_frame_{counter:06d}.jpg'), obj_mask.cpu().numpy())

    counter += 1


  patient_counter += 1

Streaming output truncated to the last 5000 lines.
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tensor([0.], device='cuda:0')
clss
tens

In [ ]:
model = YOLO('yolov8m-seg.pt')

model.tune(data='/content/gdrive/My Drive/ES100root/2024-03-14_5-Fold_Cross-val/split_5/split_5_dataset.yaml', epochs=10, iterations=100, optimizer='AdamW', plots=True, save=False, val=False)

100%|██████████| 52.4M/52.4M [00:09<00:00, 5.72MB/s]


Tuner: Initialized Tuner instance with 'tune_dir=runs/segment/tune'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/100 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
Saved runs/segment/tune/tune_scatter_plots.png
Saved runs/segment/tune/tune_fitness.png

Tuner: 1/100 iterations complete ✅ (249.19s)
Tuner: Results saved to runs/segment/tune
Tuner: Best fitness=1.23144 observed at iteration 1
Tuner: Best fitness metrics are {'metrics/precision(B)': 0.90703, 'metrics/recall(B)': 0.85214, 'metrics/mAP50(B)': 0.91931, 'metrics/mAP50-95(B)': 0.56366, 'metrics/precision(M)': 0.91532, 'metrics/recall

In [ ]:
model = YOLO('yolov8s-seg.pt')

model.tune(data='/content/gdrive/MyDrive/ES100root/2024-03-18_5-Fold_Cross-val-345789/split_3/split_3_dataset.yaml', epochs=10, iterations=30, optimizer='AdamW', plots=False, save=False, val=False)

Tuner: Initialized Tuner instance with 'tune_dir=runs/segment/tune3'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/30 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.5, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.0, 'fliplr': 0.5, 'bgr': 0.0, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}
